In [29]:
import pandas as pd

#load the cable capacities
capacitiesDf = pd.read_csv("cableCapacities.csv")
cablesDf = pd.read_csv("all_cables.csv")

print(f"Number of unique cables edges: {cablesDf.shape[0]}")
filteredCapacities = capacitiesDf[capacitiesDf["capacity"] != 0]
print(f"Number of capacity entries: {filteredCapacities.shape[0]}")
capacityNames = set(filteredCapacities["name"].unique())
capacityShortNames = set(filteredCapacities["shortName"].unique())

Number of unique cables edges: 265
Number of capacity entries: 193


In [30]:
cableNames = set(cablesDf["name"])

commonsName = capacityNames.intersection(cableNames)
print(len(commonsName))

commonsShortName = capacityShortNames.intersection(cableNames)
print(len(commonsShortName))

print(len(commonsName.union(commonsShortName)))

49
96
109


In [31]:
capacitiesDf.head()

,name,shortName,capacity
0,ACS Alaska-Oregon Network,AKRON,2621
1,Aden-Djibouti,NaN,0
2,ADRIA-1,NaN,0
3,Africa Coast to Europe Cable System,ACE,5120
4,Alaska United East,AU-East,50


In [32]:
#summary:
#   from the submatrix dataset we have 265 unique cables
#   we have 193 cables for which the capacity is known (not 0) (in the capacity dataset)
#   we can match the capacity for 109 out of the 265 cables (automatically, manually we might get more because maybe they are just named differently)


In [ ]:
#   now let's try to merge on name and short name:
import numpy as np

# First join on Name
merged1 = cablesDf.merge(filteredCapacities[["name", "capacity"]], 
                       on="name", how="left")

# Then join on Short Name (only fill NaN values from first merge)
finalDf = merged1.merge(filteredCapacities[["shortName", "capacity"]], 
                        left_on="name", right_on="shortName", 
                        how="left", suffixes=("", "_short"))

# If Capacity from Name match is missing, take Capacity from Short Name match
finalDf["capacity"] = finalDf["capacity"].fillna(finalDf["capacity_short"])

# Drop helper column
finalDf = finalDf.drop(columns=["shortName", "capacity_short"])
finalDf["capacity"] = finalDf["capacity"].fillna(0)

finalDf.to_csv("withCapacity.csv", index=False)

finalDf.head()
print(np.sum(finalDf["capacity"] != 0.0))#we have the capacities for 109 out of the 245 cables

109
